## 箱詰め問題

In [34]:
from gurobipy import *

### 問題状況の設定
-  箱詰め問題における問題状況を設定
    - ビンのサイズ
    - アイテムのサイズの種類とその数

In [35]:
def CuttingStockExample():
    """問題状況を設定"""
    B = 9   #ビンのサイズを定義
    w = [2,3,4,5,6,7,8]   #アイテムのサイズの種類を設定

    q = [4,2,6,6,2,2,2]   #各サイズのアイテム数を設定
    s = []
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    return s, B

### FFD: First Fit Decreasing
    - 大きいものから箱に詰めていく貪欲法で上限数Uを求める。
    - ヒューリスティックな解法

In [44]:
def FFD(s, B):
    """ビンの上限数Uを求める"""
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for (j, free) in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol

### モデルを定義

In [126]:
def bpp(s, B):
    # N：アイテム数　U：ビンの上限数
    N = len(s)
    U = len(FFD(s, B))
    model = Model("Bin-Packing-Problem")

    # 変数を定義
    x, y = {}, {}
    for i in range(N):
        for j in range(U):
            x[i,j] = model.addVar(vtype="B")
    for j in range(U):
        y[j] = model.addVar(vtype="B")
    model.update()

    # 制約条件を設定
    for i in range(N):
        model.addConstr(quicksum(x[i,j] for j in range(U)) == 1)
    for j in range(U):
        model.addConstr(quicksum(s[i]*x[i,j] for i in range(N)) <= B*y[j])
    for j in range(U):
        for i in range(N):
            model.addConstr(x[i,j] <= y[j])
    model.setObjective(quicksum(y[j] for j in range(U)), GRB.MINIMIZE)
    model.update()
    model.__data = x, y
    return model,x,y

### 結果を表示する関数を定義

In [135]:
def ShowResult(s, B, X):
    pos = {}
    for (i,j) in X.keys():
        if X[i,j].X == 1:
            if j not in pos:
                pos[j] = [i]
            else:
                pos[j].append(i) 
    sorted_pos = sorted(pos.items())
    for key,items in sorted_pos:
        remain = B
        print(f"ビン{str(key+1).ljust(2,' ')}の中身：",end="")
        for i, item in enumerate(items):
            remain -= s[item]
            print(f"アイテム{str(item+1).ljust(3,' ')}",end="")
        print(f"空き容量：{remain}")
            

### ビンパッキング問題を解く

In [136]:
if __name__ == "__main__":
    s, B = CuttingStockExample()
    model,x,y = bpp(s,B)
    model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 349 rows, 325 columns and 1261 nonzeros
Model fingerprint: 0x76ad1019
Variable types: 0 continuous, 325 integer (325 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 13.0000000
Presolve removed 143 rows and 0 columns
Presolve time: 0.00s
Presolved: 206 rows, 325 columns, 1794 nonzeros
Variable types: 0 continuous, 325 integer (325 binary)

Root relaxation: cutoff, 207 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        13.00000   13.00000  0.00%     -    0s

Explored 1 nodes (207

In [137]:
ShowResult(s,B,x)
    

ビン1 の中身：アイテム24 空き容量：1
ビン2 の中身：アイテム8  アイテム17 空き容量：0
ビン3 の中身：アイテム3  アイテム21 空き容量：0
ビン4 の中身：アイテム7  アイテム16 空き容量：0
ビン5 の中身：アイテム12 アイテム15 空き容量：0
ビン6 の中身：アイテム2  アイテム19 空き容量：1
ビン7 の中身：アイテム1  アイテム18 空き容量：2
ビン8 の中身：アイテム9  アイテム10 空き容量：1
ビン9 の中身：アイテム5  アイテム13 空き容量：1
ビン10の中身：アイテム6  アイテム20 空き容量：0
ビン11の中身：アイテム4  アイテム22 空き容量：0
ビン12の中身：アイテム23 空き容量：1
ビン13の中身：アイテム11 アイテム14 空き容量：0
